In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import random
import scipy
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt
import random
from scipy.stats import kendalltau
import pandas as pd

In [ ]:
adj_BA=np.load("adj_BA.npy")
adj_BA = torch.FloatTensor(adj_BA)

In [ ]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def load_data(path="./cora/", dataset="cora"):
    """读取引文网络数据cora"""
    print('Loading {} dataset...'.format(dataset))
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str)) # 使用numpy读取.txt文件
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32) # 获取特征矩阵
    labels = encode_onehot(idx_features_labels[:, -1]) # 获取标签

    # build graph
    idx = np.array(idx_features_labels[:, 0])
    #print(idx)
    idx_map = {j: i for i, j in enumerate(idx)}
    #print(idx_map)
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.dtype(str))
    #print(edges_unordered)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten()))
                     ).reshape(edges_unordered.shape)
    #print(np.where(edges==None))
    edges=np.delete(edges,[ 231,  493,  662, 1167, 1455, 2387, 2975, 3049, 3050, 3904, 3905,
       3907, 4177, 4428, 4527, 4529, 4545],axis=0)
    #print(np.where(edges==None))
    np.set_printoptions(threshold=np.inf) 
    #print(edges)
    #edges=edges.int32()
    edges=edges.astype(np.int32)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]))

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
   # adj = normalize_adj(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = torch.FloatTensor(np.array(adj.todense()))

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj,edges, features, labels, idx_train, idx_val, idx_test


In [ ]:
adj_cora, edges,features, labels, idx_train, idx_val, idx_test=load_data(path="./cora/",dataset="cora")

In [ ]:
import scanpy as sc

adata = sc.read("./jazz/jazz.mtx")

data = adata.X
adj_jazz = torch.FloatTensor(np.array(data.todense()))
adj_jazz=np.where(adj_jazz>0,1,adj_jazz)
adj_jazz=torch.FloatTensor(adj_jazz)

In [ ]:
edges=np.genfromtxt("./email-univ/email-univ.edges",
                                        dtype=np.int)
edges=edges-1
print(edges.shape)
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(1133, 1133))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj_email = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edge=np.genfromtxt("./email-dnc/email-dnc.edges",delimiter=",",
                                        dtype=np.int)
edge=edge-1
print(edge)
edges=edge[:,:2]
print(edges)
print(edges.min())
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(2029, 2029))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj_email_2 = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edge=np.genfromtxt("./moreno_oz/out.moreno_oz_oz",delimiter=" ",
                                        dtype=np.int)
edge=edge-1
print(edge)
edges=edge[:,:2]
print(edges)
print(edges.max())
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(217, 217))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj_oz = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edge=np.genfromtxt("./opsahl-usairport/out.opsahl-usairport",delimiter=" ",
                                        dtype=np.int)
edge=edge-1
print(edge)
edges=edge[:,:2]
print(edges)
print(edges.max())
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(1574, 1574))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj_usa = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edge=np.genfromtxt("./moreno_names/out.moreno_names_names",delimiter=" ",
                                        dtype=np.int)
edge=edge-1

edges=edge[:,:2]


adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(1773, 1773))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj_bible  = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./librec-filmtrust-trust/out.librec-filmtrust-trust",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(874, 874))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_ftrust = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./pajek-erdos/out.pajek-erdos",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(6927, 6927))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_erdos = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./ego-twitter/out.ego-twitter",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(23370, 23370))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_twitter = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./maayan-Stelzl/out.maayan-Stelzl",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(1706, 1706))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_Stelzl = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./maayan-figeys/out.maayan-figeys",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(2239, 2239))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_figeys = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./maayan-vidal/out.maayan-vidal",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(3133, 3133))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_vidal = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./opsahl-powergrid/out.opsahl-powergrid",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(4941, 4941))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_powergrid = torch.FloatTensor(np.array(adj.todense()))

In [ ]:
edges=np.genfromtxt("./arenas-meta/out.arenas-meta",
                                        dtype=np.int)
edges=edges-1
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(453, 453))
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

adj_Caenor = torch.FloatTensor(np.array(adj.todense()))

In [ ]:


'''
程序主要功能
输入：网络图邻接矩阵，需要被设置为感染源的节点序列，感染率，免疫率，迭代次数step
输出：被设置为感染源的节点序列的SIR感染情况---每次的迭代结果（I+R）/n
'''


def update_node_status(graph, node, beta, gamma):
    """
    更新节点状态
    :param graph: 网络图
    :param node: 节点序数
    :param beta: 感染率
    :param gamma: 免疫率
    """
    # 如果当前节点状态为 感染者(I) 有概率gamma变为 免疫者(R)
   
    # 如果当前节点状态为 易感染者(S) 有概率beta变为 感染者(I)
    if graph.nodes[node]['status'] == 'S':
        # 获取当前节点的邻居节点
        # 无向图：G.neighbors(node)
        # 有向图：G.predecessors(node)，前驱邻居节点，即指向该节点的节点；G.successors(node)，后继邻居节点，即该节点指向的节点。
        neighbors = list(graph.neighbors(node))
        # 对当前节点的邻居节点进行遍历
        for neighbor in neighbors:
            # 邻居节点中存在 感染者(I)，则该节点有概率被感染为 感染者(I)
            if graph.nodes[neighbor]['status'] == 'I':
                p = random.random()
                if p < beta:
                    graph.nodes[node]['status1'] = 'I'
                    break
    if graph.nodes[node]['status'] == 'I':
        p = random.random()
        if p < gamma:
            graph.nodes[node]['status1'] = 'R'
def update(graph):
    for node in graph:
        graph.nodes[node]['status'] = graph.nodes[node]['status1']

def count_node(graph):
    """
    计算当前图内各个状态节点的数目
    :param graph: 输入图
    :return: 各个状态（S、I、R）的节点数目
    """
    s_num, i_num, r_num = 0, 0, 0
    for node in graph:
        if graph.nodes[node]['status'] == 'S':
            s_num += 1
        elif graph.nodes[node]['status'] == 'I':
            i_num += 1
        else:
            r_num += 1
    return s_num, i_num, r_num


def SIR_network(graph, source, beta, gamma):
    """
    获得感染源的节点序列的SIR感染情况
    :param graph: networkx创建的网络
    :param source: 需要被设置为感染源的节点Id所构成的序列
    :param beta: 感染率
    :param gamma: 免疫率
    :param step: 迭代次数
    """
    n = graph.number_of_nodes()  # 网络节点个数
    sir_values = []  # 存储每一次迭代后网络中感染节点数I+免疫节点数R的总和
    # 初始化节点状态
    for node in graph:
        graph.nodes[node]['status'] = 'S'
        graph.nodes[node]['status1'] = 'S'# 将所有节点的状态设置为 易感者（S）
    # 设置初始感染源
    
    graph.nodes[source]['status'] = 'I' 
    graph.nodes[source]['status1'] = 'I'# 将感染源序列中的节点设置为感染源，状态设置为 感染者（I）
    # 记录初始状态
    sir_values.append(1 / n)
    # 开始迭代感染
    s0=n-1
    i0=1
    r0=0
    while 1:
        # 针对对每个节点进行状态更新以完成本次迭代
        for node in graph:
            update_node_status(graph, node, beta, gamma)  # 针对node号节点进行SIR过程
        update(graph)
        s, i, r = count_node(graph)  # 得到本次迭代结束后各个状态（S、I、R）的节点数目
        if s==s0 and i==i0 and r==r0:
            break
        else:
            s0=s
            i0=i
            r0=r
        sir = (i + r) / n  # 该节点的sir值为迭代结束后 感染节点数i+免疫节点数r
        sir_values.append(sir)  # 将本次迭代的sir值加入数组
    return i0+r0


In [ ]:
def caculate(adj):
    degree=np.zeros(adj.shape[0])
    for i in range(adj.shape[0]):
        degree[i]=adj[i].sum()
    
    sum_1=0
    sum_2=0
    for i in range(adj.shape[0]):
        sum_1+=degree[i]
        sum_2+=degree[i]*degree[i]
    avg1=sum_1/adj.shape[0]
    avg2=sum_2/adj.shape[0]
    
    return avg1/(avg2-avg1)

In [ ]:

def caculate_avg(graph,node):
    sum=0
    for i in range(100):
        sum+=SIR_network(graph,node,beta,1)
    return sum/100

In [ ]:
adj=np.array(adj_Stelzl)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_Stelzl=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_Stelzl.npy',lable_Stelzl)

In [ ]:
adj=np.array(adj_vidal)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_vidal=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_vidal.npy',lable_vidal)

In [ ]:
adj=np.array(adj_figeys)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_figeys=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_figeys.npy',lable_figeys)

In [ ]:
adj=np.array(adj_Caenor)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_Caenor=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_Caenor.npy',lable_Caenor)

In [ ]:
adj=np.array(adj_powergrid)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_powergrid=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_powergrid.npy',lable_powergrid)

In [ ]:

adj=np.array(adj_twitter)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_twitter=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_twitter.npy',lable_twitter)

In [ ]:
adj=np.array(adj_erdos)`
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_erdos=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_erdos.npy',lable_erdos)

In [ ]:
adj=np.array(adj_usa)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_usa=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_usa.npy',lable_usa)

In [ ]:
adj=np.array(adj_oz)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_oz=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_oz.npy',lable_oz)

In [ ]:
adj=np.array(adj_ftrust)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_ftrust=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_ftrust',lable_ftrust)

In [ ]:
adj=np.array(adj_bible)
beta=caculate(adj)*1.5
print(beta)
graph = nx.from_numpy_matrix(adj) 

lable_bible=[caculate_avg(graph,i) for i in range(adj.shape[0])]
np.save('lable_bible',lable_bible)

In [ ]:
lable_jazz=[caculate_avg(graph,i) for i in range(adj_jazz.shape[0])]
np.save('lable_jazz.npy',lable_jazz)

In [ ]:

lable_jazz=np.load("lable_jazz.npy")
lable_jazz_t=torch.tensor(lable_jazz).float()

In [ ]:
lable_twitter=np.load("lable_twitter.npy")
lable_twitter_t=torch.tensor(lable_twitter).float()

In [ ]:
lable_ftrust=np.load("lable_ftrust.npy")
lable_ftrust_t=torch.tensor(lable_ftrust).float()

In [ ]:
lable_erdos=np.load("lable_erdos.npy")
lable_erdos_t=torch.tensor(lable_erdos).float()

In [ ]:
lable_email_2=np.load("lable_email_2.npy")
lable_email_2_t=torch.tensor(lable_email_2).float()

In [ ]:
lable_email=np.load("lable_email.npy")
lable_email_t=torch.tensor(lable_email).float()

In [ ]:
lable_BA=np.load("lable_BA_1000_4.npy")
lable_BA_t=torch.tensor(lable_BA).float()

In [ ]:
lable_cora=np.load("lable.npy")
lable_cora_t=torch.tensor(lable_cora).float()

In [ ]:
lable_usa=np.load("lable_usa.npy")
lable_usa_t=torch.tensor(lable_usa).float()

In [ ]:
lable_oz=np.load("lable_oz.npy")
lable_oz_t=torch.tensor(lable_oz).float()

In [ ]:
class Den(torch.nn.Module):
    def __init__(self,n):
        super(Den,self).__init__()   
        self.w1 = nn.Parameter(torch.empty(size=(n,6)))
        self.w2= nn.Parameter(torch.empty(size=(6,n)))
        self.m=torch.nn.Softmax(dim=1)
        self.reset_parameters()
    
    def reset_parameters(self):
        #nn.init.xavier_uniform_(self.w.data,gain=1.414)
        #nn.init.xavier_uniform_(self.a.data,gain=1.414)
        for param in self.parameters():
             nn.init.xavier_uniform_(param)
    
    def forward(self,x):
        y=torch.mm(x,self.w1)
        x=torch.mm(y,self.w2)
        x=self.m(x)
        
        return y,x
        

In [ ]:
from torch_geometric.nn import GCNConv
class GNN(torch.nn.Module):
    def __init__(self,input_feature,output_feature):
        super(GNN,self).__init__()
        self.w = nn.Parameter(torch.empty(size=(input_feature,output_feature)))
        self.a= nn.Parameter(torch.empty(size=(1,output_feature)))
        self.sigmod=torch.nn.Sigmoid()
        self.reset_parameters()
    def reset_parameters(self):
        #nn.init.xavier_uniform_(self.w.data,gain=1.414)
        #nn.init.xavier_uniform_(self.a.data,gain=1.414)
        for param in self.parameters():
             nn.init.xavier_uniform_(param)
        
    def forward(self,x,adj):
        #adj=torch.Tensor(adj.numpy()+np.identity(adj.shape[0]))
        adj=torch.FloatTensor(normalize_adj(sp.csr_matrix(adj)+ sp.eye(adj.shape[0])).todense())
        x=torch.mm(adj,x)
        x=torch.mm(x,self.w)
       # x=x.add(self.a)
        x=torch.relu(x)
        return x
    
class GAE(nn.Module): #编码器
    def __init__(self, n_total_features, n_latent,p_drop=0.):
        super(GAE, self).__init__()
        self.n_total_features = n_total_features
        self.conv1 = GNN(self.n_total_features, 256)
       
        self.conv2 =GNN(128, 24)
       
        self.conv3 = GNN(256, n_latent)
        self.conv4=GNN(n_latent,self.n_total_features)
        self.fc1=torch.nn.Linear(n_latent,256)
        self.fc2=torch.nn.Linear(256,1)

    def forward(self,x,adj): #实践中一般采取多层的GCN来编码
        
        x = self.conv1(x, adj)
        #x = self.conv2(x, adj)
        x = self.conv3(x, adj) #经过三层GCN后得到节点的表示
        A = self.fc1(x) #直接算点积'
        A=self.fc2(A)
        #A = torch.sigmoid(A)
        return x,A
        



In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_BA.shape[0])).float(),adj_BA)
    
    loss_train =torch.norm(A-adj_BA.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_BA.shape[0])).float(),adj_BA)
   
    
    loss_val =torch.norm(A-adj_BA.sum(dim=1).reshape(-1,1), p='fro')
    
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_BA.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_BA,A=model(torch.tensor(np.identity(adj_BA.shape[0])).float(),adj_BA)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_erdos.shape[0])).float(),adj_erdos)
    
    loss_train =torch.norm(A-adj_erdos.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_erdos.shape[0])).float(),adj_erdos)
   
    
    loss_val =torch.norm(A-adj_erdos.sum(dim=1).reshape(-1,1), p='fro')
    
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_erdos.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(100):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_erdos,A=model(torch.tensor(np.identity(adj_erdos.shape[0])).float(),adj_erdos)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_twitter.shape[0])).float(),adj_twitter)
    
    loss_train =torch.norm(A-adj_twitter.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_twitter.shape[0])).float(),adj_twitter)
   
    
    loss_val =torch.norm(A-adj_twitter.sum(dim=1).reshape(-1,1), p='fro')
    
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_twitter.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(300):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_twitter,A=model(torch.tensor(np.identity(adj_twitter.shape[0])).float(),adj_twitter)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_ftrust.shape[0])).float(),adj_ftrust)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_ftrust.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_ftrust.shape[0])).float(),adj_ftrust)
   
    
    loss_val =  loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_ftrust.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_ftrust,A=model(torch.tensor(np.identity(adj_ftrust.shape[0])).float(),adj_ftrust)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_cora.shape[0])).float(),adj_cora)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_cora.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_cora.shape[0])).float(),adj_cora)
   
    
    loss_val =  loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_cora.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_cora,A=model(torch.tensor(np.identity(adj_cora.shape[0])).float(),adj_cora)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_email.shape[0])).float(),adj_email)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_email.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_email.shape[0])).float(),adj_email)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_email.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(400):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_email,A=model(torch.tensor(np.identity(adj_email.shape[0])).float(),adj_email)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_email_2.shape[0])).float(),adj_email_2)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_email_2.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_email_2.shape[0])).float(),adj_email_2)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_email_2.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(700):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_email_2,A=model(torch.tensor(np.identity(adj_email_2.shape[0])).float(),adj_email_2)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_jazz.shape[0])).float(),adj_jazz)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_jazz.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_jazz.shape[0])).float(),adj_jazz)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_jazz.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(700):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_jazz,A=model(torch.tensor(np.identity(adj_jazz.shape[0])).float(),adj_jazz)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_usa.shape[0])).float(),adj_usa)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_usa.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_usa.shape[0])).float(),adj_usa)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_usa.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_usa,A=model(torch.tensor(np.identity(adj_usa.shape[0])).float(),adj_usa)

In [ ]:
adj_oz=torch.tensor(adj_oz)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_oz.shape[0])).float(),adj_oz)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_oz.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_oz.shape[0])).float(),adj_oz)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_oz.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_oz,A=model(torch.tensor(np.identity(adj_oz.shape[0])).float(),adj_oz)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_Stelzl.shape[0])).float(),adj_Stelzl)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_Stelzl.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_Stelzl.shape[0])).float(),adj_Stelzl)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_Stelzl.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_Stelzl,A=model(torch.tensor(np.identity(adj_Stelzl.shape[0])).float(),adj_Stelzl)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_figeys.shape[0])).float(),adj_figeys)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_figeys.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_figeys.shape[0])).float(),adj_figeys)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_figeys.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_figeys,A=model(torch.tensor(np.identity(adj_figeys.shape[0])).float(),adj_figeys)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_vidal.shape[0])).float(),adj_vidal)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_vidal.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_vidal.shape[0])).float(),adj_vidal)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_vidal.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_vidal,A=model(torch.tensor(np.identity(adj_vidal.shape[0])).float(),adj_vidal)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_Caenor.shape[0])).float(),adj_Caenor)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_Caenor.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_Caenor.shape[0])).float(),adj_Caenor)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_Caenor.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_Caenor,A=model(torch.tensor(np.identity(adj_Caenor.shape[0])).float(),adj_Caenor)

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    
    x,A=model(torch.tensor(np.identity(adj_powergrid.shape[0])).float(),adj_powergrid)
    loss = nn.CrossEntropyLoss()
    loss_train =torch.norm(A-adj_powergrid.sum(dim=1).reshape(-1,1), p='fro')
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    x,A=model(torch.tensor(np.identity(adj_powergrid.shape[0])).float(),adj_powergrid)
   
    
    loss_val = loss_train
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature,adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model=GAE(adj_powergrid.shape[0],48)
    optimizer = optim.Adam(model.parameters(),lr= 0.01,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(500):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
            bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    #compute_test()
    model.eval()
    node_feature_powergrid,A=model(torch.tensor(np.identity(adj_powergrid.shape[0])).float(),adj_powergrid)

In [ ]:
class CNNnet(torch.nn.Module):
    def __init__(self):
        super(CNNnet,self).__init__()       
        self.conv1=torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1,
                            out_channels=10,
                            kernel_size=(3,3),
                            stride=1,
                            padding=1),
            
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,2))

        )
        self.conv2=torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=10,
                            out_channels=20,
                            kernel_size=(3,3),
                            stride=1,
                            padding=1),
            
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,2))

        )
        self.fc=torch.nn.Linear(980,6)
     
        
            
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.fc(x.view(x.size(0),-1))
        return x

class GNN(torch.nn.Module):
    def __init__(self,input_feature,output_feature):
        super(GNN,self).__init__()
        self.w = nn.Parameter(torch.empty(size=(input_feature,output_feature)))
        self.a= nn.Parameter(torch.empty(size=(1,output_feature)))
        self.sigmod=torch.nn.Sigmoid()
        self.reset_parameters()
    def reset_parameters(self):
        #nn.init.xavier_uniform_(self.w.data,gain=1.414)
        #nn.init.xavier_uniform_(self.a.data,gain=1.414)
        for param in self.parameters():
             nn.init.xavier_uniform_(param)
        
    def forward(self,x,adj):
        adj=torch.Tensor(adj.numpy()+np.identity(adj.shape[0]))
        #adj=torch.FloatTensor(normalize_adj(sp.csr_matrix(adj)+ sp.eye(adj.shape[0])).todense())
        x=torch.mm(adj,x)
        x=torch.mm(x,self.w)
        x=x.add(self.a)
        x=torch.relu(x)
        return x

class CGNN(torch.nn.Module):
    def __init__(self):
        super(CGNN,self).__init__()
        self.layer1=CNNnet()
        self.layer2=GNN(48,6)
        self.layer3=GNN(6,12)
        self.fc=torch.nn.Linear(12,1)
       
     
    
    
    def forward(self,x,adj):
        #x=self.layer1(x)
        x=self.layer2(x,adj)
        x=self.layer3(x,adj)
       # x=self.layer4(x,adj)
        
        x=self.fc(x.view(x.size(0),-1))
        x=torch.relu(x)
        x=x.flatten()
        
        return x
        

In [ ]:
class GATLayer(nn.Module):
    """GAT层"""
    def __init__(self,input_feature,output_feature,dropout,alpha,concat=True):
        super(GATLayer,self).__init__()
        self.input_feature = input_feature
        self.output_feature = output_feature
        self.alpha = alpha
        self.dropout = dropout
        self.concat = concat
        self.a = nn.Parameter(torch.empty(size=(2*output_feature,1)))
        self.w = nn.Parameter(torch.empty(size=(input_feature,output_feature)))
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.reset_parameters()
    
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.w.data,gain=1.414)
        nn.init.xavier_uniform_(self.a.data,gain=1.414)
    
    def forward(self,h,adj):
        Wh = torch.mm(h,self.w)
        e = self._prepare_attentional_mechanism_input(Wh)
        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec) # adj>0的位置使用e对应位置的值替换，其余都为-9e15，这样设定经过Softmax后每个节点对应的行非邻居都会变为0。
        attention = F.softmax(attention, dim=1) # 每行做Softmax，相当于每个节点做softmax
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.mm(attention, Wh) # 得到下一层的输入
        
        if self.concat:
            return F.relu(h_prime) #激活
        else:
            return h_prime
        
    def _prepare_attentional_mechanism_input(self,Wh):
        
        Wh1 = torch.matmul(Wh,self.a[:self.output_feature,:]) # N*out_size @ out_size*1 = N*1
        
        Wh2 = torch.matmul(Wh,self.a[self.output_feature:,:]) # N*1
        
        e = Wh1+Wh2.T # Wh1的每个原始与Wh2的所有元素相加，生成N*N的矩阵
        return self.leakyrelu(e)
    
class GAT(nn.Module):

    def __init__(self,input_size,hidden_size,output_size,dropout,alpha,nheads,concat=True):
        super(GAT,self).__init__()
        self.dropout= dropout
        self.attention = [GATLayer(input_size, hidden_size, dropout=dropout, alpha=alpha,concat=True) for _ in range(nheads)]
        for i,attention in enumerate(self.attention):
            self.add_module('attention_{}'.format(i),attention)
        
        self.out_att = GATLayer(hidden_size*nheads, output_size, dropout=dropout, alpha=alpha,concat=False)
        self.fc=torch.nn.Linear(28,1)
        #self.layer2=GNN(28,128)
        
    def forward(self,x,adj):
        #x = F.dropout(x,self.dropout,training=self.training)
        #x = torch.cat([att(x,adj) for att in self.attention],dim=1)
        #x = F.dropout(x,self.dropout,training=self.training)
       # x = self.out_att(x,adj)
        #x=self.layer2(x,adj)
        x.view(x.size(0),-1)
        x=self.fc(x.view(x.size(0),-1))
        x=torch.relu(x)
        x=x.flatten()
        
        return x


In [ ]:
class RCNN(torch.nn.Module):
    def __init__(self):
        super(RCNN,self).__init__()       
        self.conv1=torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1,
                            out_channels=16,
                            kernel_size=(5,5),
                            stride=1,
                            padding=2),
            
            torch.nn.LeakyReLU()
            

        )
        self.conv2=torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16,
                            out_channels=32,
                            kernel_size=(5,5),
                            stride=1,
                            padding=2),
            
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(kernel_size=(2,2)),
            torch.nn.MaxPool2d(kernel_size=(2,2))

        )
        self.fc=torch.nn.Linear(1568,1)
        
    def forward(self,x):
            x=self.conv1(x)
            x=self.conv2(x)
            x=self.fc(x.view(x.size(0),-1))
            x=torch.relu(x)
            x=x.flatten()
            return x

In [ ]:
label_t=lable_BA_t
node_feature=node_feature_BA.data
adj=adj_BA

In [ ]:
def MI(res):
    a=pd.DataFrame(res.detach().numpy())
    x=len(a)
    a.rank(axis=0,method='min',numeric_only=None,
    na_option='keep',ascending=True,pct=False)
    b=a.iloc[:,0].value_counts()
    y=0
    for i in range(len(b)):
        y=y+b.iloc[i]*(b.iloc[i]-1)
    ans=(1-y/(x*(x-1)))*(1-y/(x*(x-1)))
    return ans

In [ ]:
class ListMLE(nn.Module):
    def __init__(self):
        super(ListMLE, self).__init__()
    def forward(self, outputs, labels):
        scores = torch.zeros_like(outputs)
        for t in range(scores.size()[0]):
            scores[t] = torch.logcumsumexp(outputs[t, labels[t]], dim=0)
        loss = torch.mean(scores - outputs)
        return loss

In [ ]:
list_loss=ListMLE()

In [ ]:
import torch

#DEFAULT_EPS = 1e-10
DEFAULT_EPS = 0.0001
PADDED_Y_VALUE = -1
PADDED_INDEX_VALUE = -1

def listMLE(y_pred, y_true, eps=DEFAULT_EPS, padded_value_indicator=PADDED_Y_VALUE):
    """
    FROM: https://github.com/allegro/allRank/blob/master/allrank/models/losses/listMLE.py
    ListMLE loss introduced in "Listwise Approach to Learning to Rank - Theory and Algorithm".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: loss value, a torch.Tensor
    """
    # Reshape the input
    #if len(y_true.size()) == 1:
    y_pred = y_pred.view(1, -1)
    y_true = y_true.view(1, -1)
    print('listmle: y_true:', y_true.size(), 'y_pred', y_pred.size())
    # shuffle for randomised tie resolution
    random_indices = range(y_pred.shape[-1])
    print(y_pred.shape[-1])
    y_pred_shuffled = y_pred[:, random_indices]
    y_true_shuffled = y_true[:, random_indices]

    y_true_sorted, indices = y_true_shuffled.sort(descending=True, dim=-1)

    mask = y_true_sorted == padded_value_indicator

    preds_sorted_by_true = torch.gather(y_pred_shuffled, dim=1, index=indices)
    preds_sorted_by_true[mask] = float("-inf")

    max_pred_values, _ = preds_sorted_by_true.max(dim=1, keepdim=True)

    preds_sorted_by_true_minus_max = preds_sorted_by_true - max_pred_values

    cumsums = torch.cumsum(preds_sorted_by_true_minus_max.exp().flip(dims=[1]), dim=1).flip(dims=[1])

    observation_loss = torch.log(cumsums + eps) - preds_sorted_by_true_minus_max
    #print('listmle obs loss:', observation_loss)

    observation_loss[mask] = 0.0
    listmle = torch.mean(torch.sum(observation_loss, dim=1))
    print('listmle loss:', listmle.item())

    return listmle

def train(epoch):
    t = time.time()
    model.train()
    
    output = model(node_feature,adj)
    loss_train = listMLE(output, label_t)
    #loss_train = torch.nn.functional.mse_loss(output, label_t)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model.eval()
    output = model(node_feature, adj)
   
    
    loss_val = torch.nn.functional.mse_loss(output, label_t)
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {}'.format(loss_train.data.item()),
          
          'loss_val: {}'.format(loss_val.data.item()),
        
          'time: {}s'.format(time.time() - t))
    return loss_train.data.item()

def compute_test():
    model.eval()
    output = model(node_feature, adj)
    loss_test =torch.nn.functional.mse_loss(output, label_t)
  
    
    print("Test set results:",
          "loss= {}".format(loss_test.data.item()),
          )

if __name__ == '__main__':

    
    random.seed(17)
    np.random.seed(17)
    torch.manual_seed(17)
    
    model = CGNN()
    optimizer = optim.Adam(model.parameters(),lr=0.001,weight_decay=5e-4)
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 1000+1
    best_epoch = 0


    for epoch in range(3000):
        loss_values.append(train(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
        
        if bad_counter == 100:
             bad_counter += 1
    print("Optimization Finished!")
    print("Total time elapsed: {}s".format(time.time() - t_total))
    compute_test()

In [ ]:
lable_Stelzl=np.load("lable_Stelzl.npy")
lable_Stelzl_t=torch.tensor(lable_Stelzl).float()

lable_figeys=np.load("lable_figeys.npy")
lable_figeys_t=torch.tensor(lable_figeys).float()

lable_vidal=np.load("lable_vidal.npy")
lable_vidal_t=torch.tensor(lable_vidal).float()

lable_Caenor=np.load("lable_Caenor.npy")
lable_Caenor_t=torch.tensor(lable_Caenor).float()

lable_powergrid=np.load("lable_powergrid.npy")
lable_powergrid_t=torch.tensor(lable_powergrid).float()

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_email,adj_email)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_email_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_email_t))
print(MI(output))
#np.save("output_agnn_email",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_email_2,adj_email_2)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_email_2_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_email_2_t))
print(MI(output))
#np.save("output_agnn_email_2",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_cora,adj_cora)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_cora_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_cora_t))
print(MI(output))
#np.save("output_agnn_cora",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_jazz,adj_jazz)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_jazz_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_jazz_t))
print(MI(output))
#np.save("output_agnn_jazz",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_usa,adj_usa)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_usa_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_usa_t))
print(MI(output))
#np.save("output_agnn_usa",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_oz,adj_oz)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_oz_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_oz_t))
print(MI(output))
#np.save("output_agnn_oz",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_Stelzl,adj_Stelzl)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_Stelzl_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_Stelzl_t))
print(MI(output))
print(end-start)
np.save("output_agnn_Stelzl",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_figeys,adj_figeys)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_figeys_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_figeys_t))
print(MI(output))
print(end-start)
np.save("output_agnn_figeys",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_vidal,adj_vidal)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_vidal_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_vidal_t))
print(MI(output))
print(end-start)
np.save("output_agnn_vidal",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_Caenor,adj_Caenor)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_Caenor_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_Caenor_t))
print(MI(output))
print(end-start)
np.save("output_agnn_Caenor",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_powergrid,adj_powergrid)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_powergrid_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_powergrid_t))
print(MI(output))
print(end-start)
np.save("output_agnn_powergrid",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_twitter,adj_twitter)
end=time.time()
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_twitter_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_twitter_t))
print(MI(output))
print(end-start)
#np.save("output_agnn_twitter",output.detach().numpy())

In [ ]:

model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_erdos,adj_erdos)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_erdos_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_erdos_t))
print(MI(output))


In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_ftrust,adj_ftrust)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_ftrust_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_ftrust_t))
print(MI(output))
#np.save("output_agnn_ftrust",output.detach().numpy())

In [ ]:
model_cgnn=model
model_cgnn.eval()
start=time.time()
output = model_cgnn(node_feature_bible,adj_bible)
end=time.time()
print(end-start)
output_rank=output.detach().numpy().argsort()

#print(output_rank)
#print(label_rank)
loss_train = torch.nn.functional.mse_loss(output,lable_bible_t)
print(loss_train)
print(kendalltau(output.detach().numpy(),lable_bible_t))
print(MI(output))
#np.save("output_agnn_bible",output.detach().numpy())